In [1]:
from accessKeys import API_KEY
import requests
import json
import pandas as pd
# import seaborn as sns
# from matplotlib import pyplot as plt
# import boto3

from bs4 import BeautifulSoup

import time
%matplotlib inline  
sns.set()
plt.style.use('fivethirtyeight')

In [2]:
def get_articles_json(month, year, api_key=API_KEY):
    print('Getting articles for {}/{}'.format(month,year))
    url = 'https://api.nytimes.com/svc/archive/v1/{}/{}.json'.format(year, month)
    params = {'api-key': api_key}
    response = requests.get(url, params=params)
    print('status code: {}'.format(response.status_code))
    while response.status_code != 200:
        print('trying again...')
        time.sleep(15)
        response = requests.get(url, params=params)
        print('status code: {}'.format(response.status_code))
    data = json.loads(response.text)
    try:
        data['response']['docs']
        print('Get articles success')
        return data['response']['docs']
    except Exception as e:
        print('ERROR:, {}'.format(e))
        return {}

In [23]:
# import time

def get_year(x):
    return x['pub_date'].split('T')[0].split('-')[0]

def get_month(x):
    return x['pub_date'].split('T')[0].split('-')[1]

def clean_df(df_raw):
    temp_df = df_raw[df_raw['type_of_material'] == 'News'].reset_index()
    temp_df['year'] = temp_df.apply(get_year, axis=1)
    temp_df['month'] = temp_df.apply(get_month, axis=1)
    temp_df = temp_df[['year', 'month', '_id', 'web_url', 'word_count']]
    return temp_df

def parse_url(url, retry=False):
    time.sleep(1)
    response_text = requests.get(url)    
    soup = BeautifulSoup(response_text.content, 'html.parser')
    all_text = soup.find_all(attrs={'class':'story-body-text'})
    if len(all_text) == 0 and not retry:
        joined_text = parse_url(url, retry=True)
    else:
        joined_text = ' '.join([x.text for x in all_text])
    return joined_text

def get_write_article(year, month, start_point=0):
    raw_df = pd.DataFrame(get_articles_json(month,year))
    cleaned_df = clean_df(raw_df)
    print('number of articles: {}'.format(len(cleaned_df)))
    n = 50  #chunk row size
    # Split dataframe into chunks so we can parse and write to csv in chunks
    temp_dfs = [cleaned_df[i:i+n].copy() for i in range(start_point,cleaned_df.shape[0],n)]
    
    with open('{}_{}.csv'.format(year, month), 'a') as f:
        for idx, df_chunk in enumerate(temp_dfs):
            df_chunk['article'] = df_chunk['web_url'].apply(parse_url)
            df_chunk.to_csv(f, header=False, encoding='utf-8')
            print('done writing lines {}'.format((idx + 1) * n))
#             if idx >= 100:
#                 return
            time.sleep(5)

In [24]:
for i in range(6,13):
    get_write_article(2017,i)
    time.sleep(30)

Getting articles for 2/2017
status code: 200
Get articles success
number of articles: 3216
done writing lines 10
done writing lines 20
done writing lines 30
done writing lines 40
done writing lines 50
done writing lines 60
done writing lines 70
done writing lines 80
done writing lines 90
done writing lines 100
done writing lines 110
done writing lines 120
done writing lines 130
done writing lines 140
done writing lines 150
done writing lines 160
done writing lines 170
done writing lines 180
done writing lines 190
done writing lines 200
done writing lines 210
done writing lines 220
done writing lines 230
done writing lines 240
done writing lines 250
done writing lines 260
done writing lines 270
done writing lines 280
done writing lines 290
done writing lines 300
done writing lines 310
done writing lines 320
done writing lines 330
done writing lines 340
done writing lines 350
done writing lines 360
done writing lines 370
done writing lines 380
done writing lines 390
done writing lines 40

done writing lines 400
done writing lines 410
done writing lines 420
done writing lines 430
done writing lines 440
done writing lines 450
done writing lines 460
done writing lines 470
done writing lines 480
done writing lines 490
done writing lines 500
done writing lines 510
done writing lines 520
done writing lines 530
done writing lines 540
done writing lines 550
done writing lines 560
done writing lines 570
done writing lines 580
done writing lines 590
done writing lines 600
done writing lines 610
done writing lines 620
done writing lines 630
done writing lines 640
done writing lines 650
done writing lines 660
done writing lines 670
done writing lines 680
done writing lines 690
done writing lines 700
done writing lines 710
done writing lines 720
done writing lines 730
done writing lines 740
done writing lines 750
done writing lines 760
done writing lines 770
done writing lines 780
done writing lines 790
done writing lines 800
done writing lines 810
done writing lines 820
done writin

KeyboardInterrupt: 

In [ ]:
for i in range(1,13):
    get_write_article(2016,i,start_point=14)
    time.sleep(30)

In [ ]:
for i in range(1,13):
    get_write_article(2015,i,start_point=14)
    time.sleep(30)

In [ ]:
for i in range(1,13):
    get_write_article(2014,i,start_point=10)
    time.sleep(60)